- 근처 학교와 떨어져 있는 평균 거리 (distance)(쭈니)
- 가장 가까운 학교와의 거리
- 같은 시도에 있는 학교 수
- 해당 학교의 첫 유저 가입 시기 (연도-월)

In [1]:
import pandas as pd
import os

# 데이터 디렉토리
data_dir = "/home/jovyan/work/data"

all_data = []

# 각 테이블에서 데이터 추출
tables = [
    ('accounts_blockrecord_processed.csv', 'user_id'),
    ('accounts_failpaymenthistory.csv', 'user_id'),
    ('accounts_paymenthistory.csv', 'user_id'),
    ('accounts_pointhistory.csv', 'user_id'),
    ('accounts_timelinereport.csv', 'user_id'),
    ('accounts_user_processed.csv', 'id'),  # id가 user_id 역할
    ('accounts_userquestionrecord_processed.csv', 'user_id'),
    ('event_receipts.csv', 'user_id'),
    ('polls_questionreport.csv', 'user_id'),
    ('polls_questionset.csv', 'user_id')
]

# 일반 테이블들 처리
for file_name, user_col in tables:
    df = pd.read_csv(os.path.join(data_dir, file_name))
    temp_df = df[[user_col, 'created_at']].copy()
    temp_df.columns = ['user_id', 'created_at']
    temp_df['table_name'] = file_name.replace('.csv', '')
    all_data.append(temp_df)

# 친구 요청 테이블 - 보낸 사람만 포함 (능동적 활동)
df_friend = pd.read_csv(os.path.join(data_dir, 'accounts_friendrequest.csv'))

# 보낸 사람만 (능동적 활동)
temp_send = df_friend[['send_user_id', 'created_at']].copy()
temp_send.columns = ['user_id', 'created_at']
temp_send['table_name'] = 'accounts_friendrequest_send'
all_data.append(temp_send)

# 모든 데이터 병합
merged_df = pd.concat(all_data, ignore_index=True)

# created_at을 datetime으로 변환
merged_df['created_at'] = pd.to_datetime(merged_df['created_at'], format='ISO8601', errors='coerce')

# null 값 제거
merged_df = merged_df.dropna(subset=['created_at'])

# user_id로 그룹화하고 created_at으로 정렬
merged_df = merged_df.sort_values(['user_id', 'created_at']).reset_index(drop=True)

print(f"총 {len(merged_df):,}개 레코드 병합 완료")
print(f"유니크 사용자 수: {merged_df['user_id'].nunique():,}")
print("\n테이블별 레코드 수:")
print(merged_df['table_name'].value_counts())

총 21,705,813개 레코드 병합 완료
유니크 사용자 수: 677,085

테이블별 레코드 수:
table_name
accounts_friendrequest_send              17147175
accounts_pointhistory                     2338918
accounts_userquestionrecord_processed     1217558
accounts_user_processed                    677085
polls_questionset                          158384
accounts_paymenthistory                     95140
polls_questionreport                        51424
accounts_blockrecord_processed              19449
event_receipts                                309
accounts_timelinereport                       208
accounts_failpaymenthistory                   163
Name: count, dtype: int64


In [24]:
import pandas as pd
import os

# 경로 설정
data_dir = '/home/jovyan/work/data'

# 학급 테이블
accounts_group = pd.read_csv(os.path.join(data_dir, 'accounts_group.csv'))

# 가까운 학교를 기록해두기 위한 관계형 테이블
accounts_nearbyschool = pd.read_csv(os.path.join(data_dir, 'accounts_nearbyschool.csv'))

# 유저 컨택 테이블
accounts_user_contacts = pd.read_csv(os.path.join(data_dir, 'accounts_user_contacts.csv'))

# 학교 테이블
accounts_school = pd.read_csv(os.path.join(data_dir, 'accounts_school.csv'))

# 유저 테이블
accounts_user_processed = pd.read_csv(os.path.join(data_dir, 'accounts_user_processed.csv'))


# 근처 학교와 떨어져 있는 평균 거리(distance)

In [25]:
accounts_nearbyschool

,id,distance,nearby_school_id,school_id
0,119021,0.004564,6,7
1,119022,0.010787,13,7
2,119023,0.012928,20,7
3,119024,0.013590,4,7
4,119025,0.014122,24,7
...,...,...,...,...
59495,178516,0.018277,1498,5961
59496,178517,0.018652,1474,5961
59497,178518,0.021143,1466,5961
59498,178519,0.024572,1467,5961


In [26]:
# 학교별 근처 학교 평균 거리 계산
avg_distance = (
    accounts_nearbyschool
    .groupby('school_id')['distance']
    .mean()
    .reset_index()
    .rename(columns={'distance': 'avg_distance_to_nearby_schools'})
)

avg_distance


,school_id,avg_distance_to_nearby_schools
0,4,0.012985
1,5,0.025214
2,6,0.011439
3,7,0.012182
4,8,0.021749
...,...,...
5945,5959,0.083070
5946,5960,0.043054
5947,5961,0.014855
5948,5963,0.054355


# 가장 가까운 학교와의 거리

In [27]:
# 자기 자신을 제외하고 가장 가까운 학교 거리 계산
min_distance = (
    accounts_nearbyschool[accounts_nearbyschool['school_id'] != accounts_nearbyschool['nearby_school_id']]
    .groupby('school_id')['distance']
    .min()
    .reset_index()
    .rename(columns={'distance': 'min_distance_to_school'})
)

min_distance

,school_id,min_distance_to_school
0,4,0.007764
1,5,0.010535
2,6,0.004564
3,7,0.004564
4,8,0.004013
...,...,...
5945,5959,0.004583
5946,5960,0.026451
5947,5961,0.010005
5948,5963,0.046101


In [28]:
min_distance['min_distance_to_school'].unique()

array([0.00776373, 0.01053466, 0.00456357, ..., 0.01000533, 0.04610105,
       0.03915378])

# 같은 시도에 있는 학교 수

In [29]:
accounts_school

,id,address,student_count,school_type
0,4,충청북도 충주시,239,H
1,5,충청북도 충주시,160,M
2,6,충청북도 충주시,200,H
3,7,충청북도 충주시,114,H
4,8,충청북도 충주시,139,M
...,...,...,...,...
5946,5960,경상남도 양산시,166,H
5947,5961,인천광역시 서구,77,M
5948,5963,경북 김천시,17,H
5949,5964,-,1,H


In [30]:
# address가 이상한 행 제거
accounts_school = accounts_school[accounts_school['address'] != '-'].copy()

In [31]:
# 같은 address를 공유하는 학교 수
accounts_school['num_schools_with_same_address'] = (
    accounts_school.groupby('address')['id'].transform('count')
)

In [32]:
accounts_school

,id,address,student_count,school_type,num_schools_with_same_address
0,4,충청북도 충주시,239,H,34
1,5,충청북도 충주시,160,M,34
2,6,충청북도 충주시,200,H,34
3,7,충청북도 충주시,114,H,34
4,8,충청북도 충주시,139,M,34
...,...,...,...,...,...
5945,5959,제주특별자치도 서귀포시,3,H,28
5946,5960,경상남도 양산시,166,H,29
5947,5961,인천광역시 서구,77,M,47
5948,5963,경북 김천시,17,H,1


In [33]:
# 이상한 address 값 확인
invalid_addresses = accounts_school[accounts_school['address'].isin(['-', '', None])]
print(invalid_addresses)
print(f'이상한 address를 가진 학교 수: {len(invalid_addresses)}')


Empty DataFrame
Columns: [id, address, student_count, school_type, num_schools_with_same_address]
Index: []
이상한 address를 가진 학교 수: 0


# 각 학교의 첫 유저 가입 시기(연도-월)

In [34]:
accounts_user_processed

,id,is_superuser,is_staff,gender,point,friend_id_list,is_push_on,created_at,block_user_id_list,hide_user_id_list,...,report_count,alarm_count,pending_chat,pending_votes,group_id,friend_count,is_point_specialist,is_friend_specialist,is_specialist,specialist_type
0,831956,1,1,NaN,600,"[1292473, 913158, 1488461, 1064695, 1043565, 1...",0,2023-03-29 03:44:14.047130,[],[],...,0,0,0,0,NaN,6,False,False,False,normal
1,831962,0,0,F,2248,"[833025, 832642, 982531, 879496, 838541, 83752...",1,2023-03-29 05:18:56.162368,[],[],...,253,40878,5499,110,12.0,43,False,False,False,normal
2,832151,0,0,M,1519,"[838785, 982531, 882567, 879496, 838541, 83649...",0,2023-03-29 12:56:34.989468,[],[],...,0,37,0,47,1.0,51,False,False,False,normal
3,832340,0,0,F,57,"[841345, 982531, 838785, 963714, 882567, 83252...",1,2023-03-29 12:56:35.020790,[],[],...,0,19,0,21,1.0,57,False,False,False,normal
4,832520,0,0,M,1039,"[874050, 849763, 874212, 844297, 838541, 84004...",0,2023-03-29 12:56:35.049311,[],[],...,0,29,0,15,12.0,18,False,False,False,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
677080,1583729,0,0,M,300,[],1,2024-05-08 21:54:33.621408,[],[],...,0,0,0,0,32442.0,0,False,False,False,normal
677081,1583730,0,0,M,420,"[1109507, 1110921, 1111072, 1110054, 1108393, ...",0,2024-05-09 07:08:11.001817,[],[],...,0,1,0,0,43949.0,21,False,False,False,normal
677082,1583731,0,0,M,300,"[1583732, 1583673]",1,2024-05-09 07:22:19.186439,[],[],...,0,1,0,0,18640.0,2,False,False,False,normal
677083,1583732,0,0,F,300,"[1583673, 1583730, 1583731]",1,2024-05-09 07:22:38.387553,[],[],...,0,0,0,0,18640.0,3,False,False,False,normal


In [35]:
accounts_group

,id,grade,class_num,school_id
0,1,1,1,1
1,8,3,2,1
2,9,3,3,1
3,10,2,2,1
4,11,2,3,1
...,...,...,...,...
84510,84542,3,2,5107
84511,84543,2,7,3487
84512,84544,3,1,5043
84513,84545,2,7,416


In [36]:
accounts_user_contacts

,id,contacts_count,invite_user_id_list,user_id
0,259,30,[],1167696
1,1756,79,[],863169
2,13742,21,[854615],857205
3,13754,29,[],851431
4,13756,28,[849318],855476
...,...,...,...,...
5058,12981327,7,[],1480714
5059,13391623,1,[],1506575
5060,14465598,0,[],1577436
5061,14579987,0,[],1582145


In [37]:
merged_df

,user_id,created_at,table_name
0,831956,2023-03-29 03:44:14.047130,accounts_user_processed
1,831962,2023-03-29 05:18:56.162368,accounts_user_processed
2,831962,2023-07-13 07:51:59.000000,accounts_friendrequest_send
3,832151,2023-03-29 12:56:34.989468,accounts_user_processed
4,832151,2023-04-22 06:02:56.000000,accounts_friendrequest_send
...,...,...,...
21705808,1583732,2024-05-09 07:22:38.387553,accounts_user_processed
21705809,1583732,2024-05-09 07:23:12.000000,accounts_friendrequest_send
21705810,1583732,2024-05-09 07:23:12.000000,accounts_friendrequest_send
21705811,1583732,2024-05-09 07:23:12.000000,accounts_friendrequest_send


In [38]:
# group_id가 NaN인 유저 수
null_group_users = accounts_user_processed['group_id'].isna().sum()

print(f'group_id가 없는 유저 수: {null_group_users}명')


group_id가 없는 유저 수: 3명


In [39]:
# 1. user_id → group_id (NaN 제거 + int 변환)
user_group = (
    accounts_user_processed[['id', 'group_id']]
    .rename(columns={'id': 'user_id'})
    .dropna(subset=['group_id'])  # group_id가 없는 유저 제외
)

# group_id를 int로 변환 (accounts_group과 병합하기 위해)
user_group['group_id'] = user_group['group_id'].astype(int)

# 2. group_id → school_id
group_school = (
    accounts_group[['id', 'school_id']]
    .rename(columns={'id': 'group_id'})
)

# 3. user_id → 가입 시기 (가입 로그 중 accounts_user_processed만)
user_signup_time = merged_df[
    merged_df['table_name'] == 'accounts_user_processed'
][['user_id', 'created_at']]

# 4. 병합: 유저 ↔ 그룹 ↔ 학교 ↔ 가입 시기
user_school_signup = (
    user_group
    .merge(group_school, on='group_id', how='left')
    .merge(user_signup_time, on='user_id', how='left')
)

# 5. 가입 시기를 연-월로 변환
user_school_signup['signup_month'] = pd.to_datetime(user_school_signup['created_at']).dt.to_period('M').astype(str)

# 6. 학교별 첫 유저 가입 시기 계산
first_signup_month = (
    user_school_signup
    .groupby('school_id')['signup_month']
    .min()
    .reset_index()
    .rename(columns={'signup_month': 'first_user_signup_month'})
)

In [40]:
first_signup_month

,school_id,first_user_signup_month
0,1,2023-03
1,4,2023-05
2,5,2023-05
3,6,2023-05
4,7,2023-04
...,...,...
5546,5959,2023-05
5547,5960,2023-05
5548,5961,2023-05
5549,5963,2023-05


In [41]:
user_count = user_school_signup['user_id'].nunique()
print(f'유저 수: {user_count}')


유저 수: 677082


In [42]:
school_count = user_school_signup['school_id'].nunique()
print(f'유저가 속한 학교 수: {school_count}')



유저가 속한 학교 수: 5551


In [43]:
# user_school_signup 기준으로 school_id 추출
school_ids_with_users = set(user_school_signup['school_id'].dropna().unique())

# first_signup_month 기준 school_id
school_ids_with_signup = set(first_signup_month['school_id'].unique())

# 유저는 있었는데 first_signup_month에 없는 school_id
missing_in_signup = school_ids_with_users - school_ids_with_signup
print(f'유저는 있었는데 first_signup_month에 빠진 학교 수: {len(missing_in_signup)}')
print(missing_in_signup)


유저는 있었는데 first_signup_month에 빠진 학교 수: 0
set()


# 병합

In [54]:
# 'id' → 'school_id'로 변경
accounts_school_renamed = accounts_school.rename(columns={'id': 'school_id'})

# 병합
school_features = accounts_school_renamed.merge(first_signup_month, on='school_id', how='left')


In [55]:
# 평균 거리와 최소 거리 피처 병합
school_features = (
    school_features
    .merge(avg_distance, on='school_id', how='left')
    .merge(min_distance, on='school_id', how='left')
)

In [56]:
# 누락된 거리 정보 확인
missing_avg = school_features['avg_distance_to_nearby_schools'].isna().sum()
missing_min = school_features['min_distance_to_school'].isna().sum()

print(f'평균 거리 누락: {missing_avg}개')
print(f'최소 거리 누락: {missing_min}개')


평균 거리 누락: 1개
최소 거리 누락: 1개


In [57]:
school_features

,school_id,address,student_count,school_type,num_schools_with_same_address,first_user_signup_month,avg_distance_to_nearby_schools,min_distance_to_school
0,4,충청북도 충주시,239,H,34,2023-05,0.012985,0.007764
1,5,충청북도 충주시,160,M,34,2023-05,0.025214,0.010535
2,6,충청북도 충주시,200,H,34,2023-05,0.011439,0.004564
3,7,충청북도 충주시,114,H,34,2023-04,0.012182,0.004564
4,8,충청북도 충주시,139,M,34,2023-05,0.021749,0.004013
...,...,...,...,...,...,...,...,...
5943,5959,제주특별자치도 서귀포시,3,H,28,2023-05,0.083070,0.004583
5944,5960,경상남도 양산시,166,H,29,2023-05,0.043054,0.026451
5945,5961,인천광역시 서구,77,M,47,2023-05,0.014855,0.010005
5946,5963,경북 김천시,17,H,1,2023-05,0.054355,0.046101


In [58]:
# 평균 거리 또는 최소 거리가 NaN인 행 수 확인
missing_rows = school_features[
    school_features['avg_distance_to_nearby_schools'].isna() |
    school_features['min_distance_to_school'].isna()
]

print(f'결측치가 있는 행 수: {len(missing_rows)}')


결측치가 있는 행 수: 1


In [60]:
# school_features를 Parquet 파일로 저장
school_features.to_parquet('school_features.parquet', index=False, engine='pyarrow')


In [62]:
# 필요한 컬럼 리스트
selected_columns = [
    'school_id',
    'student_count',
    'num_schools_with_same_address',
    'avg_distance_to_nearby_schools',
    'min_distance_to_school'
]

# 추출 후 저장
school_features[selected_columns].to_parquet(
    'school_features.parquet', index=False
)


In [63]:
school_features.to_parquet('gs://sprintda05_final_project/machine_learning/school/school_features.parquet', index=False, engine='pyarrow')
